In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

sns.set()

In [24]:
gdp = pd.read_csv('gdp-per-capita.csv')
employment_above_15 = pd.read_csv('employment-above-15.csv')
male_completion = pd.read_csv('male-completion-rate.csv')
female_completion = pd.read_csv('female-completion-rate.csv')

countries = np.array(employment_above_15['Country'])
employment_1991 = np.array(employment_above_15['1991'])
employment_2000 = np.array(employment_above_15['2000'])

In [22]:
def max_employment(countries, employment):
    i = employment.argmax()
    return countries[i], employment[i]

In [23]:
print max_employment(countries, employment_2000)

('Tanzania', 85.099998474100005)
